In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ExponentialLR

In [26]:
# Download training data from open datasets.
# Guassian blur - how to do: RESEARCH
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.RandomHorizontalFlip(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomVerticalFlip(), transforms.RandomRotation(45),
                                    transforms.GaussianBlur(3), transforms.Pad(4)]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                   transforms.Pad(4)]),
)

In [27]:
batch_size = 32
num_classes = 102
learning_rate = 0.001
num_epochs = 100

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [28]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=128) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=512)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.conv_layer6 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=512)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(12800 , 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.relu1(self.conv_layer1(x))
        out = self.relu1(self.conv_layer2(out))
        out = self.batch_layer1(out)
        out = self.max_pool1(out)
        
        out = self.relu1(self.conv_layer3(out))
        out = self.relu1(self.conv_layer4(out))
        out = self.batch_layer2(out)
        out = self.max_pool2(out)

        out = self.relu1(self.conv_layer5(out))
        out = self.relu1(self.conv_layer6(out))
        out = self.batch_layer3(out)
        out = self.max_pool3(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)

def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join(save_filename)
    torch.save(network.state_dict(), save_path)

In [29]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
scheduler = ExponentialLR(optimizer, 0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
saveCounter = 0
minLoss = 10000

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if minLoss > loss:
        saveCounter += 1
        save_network(net, saveCounter)
        minLoss = loss

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/100], Loss: 4.4195
Epoch [2/100], Loss: 3.6959
Epoch [3/100], Loss: 3.4746
Epoch [4/100], Loss: 3.0762
Epoch [5/100], Loss: 3.0764
Epoch [6/100], Loss: 2.9923
Epoch [7/100], Loss: 2.4648
Epoch [8/100], Loss: 2.5478
Epoch [9/100], Loss: 2.3204
Epoch [10/100], Loss: 2.0749
Epoch [11/100], Loss: 2.4258
Epoch [12/100], Loss: 2.0073
Epoch [13/100], Loss: 1.4718
Epoch [14/100], Loss: 1.4372
Epoch [15/100], Loss: 1.2303
Epoch [16/100], Loss: 1.3208
Epoch [17/100], Loss: 1.0070
Epoch [18/100], Loss: 0.9853
Epoch [19/100], Loss: 0.8486
Epoch [20/100], Loss: 0.5415
Epoch [21/100], Loss: 0.9205


In [ ]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))